In [22]:
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import nltk
import re
import numpy as np

In [127]:
codes = pd.read_csv("../processed_data/code_top300.csv")
# codes.iloc[:,0] = codes.index.values
df = pd.DataFrame(columns = ['text', 'tokens', 'annotations'])

In [38]:
data = np.zeros((1,3))
data2 = np.ones((1,3))

In [39]:
df = df.append(pd.DataFrame(data, columns = ['text', 'tokens', 'annotations']))

In [40]:
df.append(pd.DataFrame(data2, columns = ['text', 'tokens', 'annotations']))

,text,tokens,annotations
0,0.0,0.0,0.0
0,1.0,1.0,1.0


In [3]:
identifier = r'[a-zA-Z_][\d\w_]*'

In [4]:
keywords = r'''break|default|func|interface|select|case|defer
                |go|map|struct|chan|else|goto|package|switch
                |const|fallthrough|if|range|type|continue|for|import|return|var'''

In [5]:
# operators and punctuation
operators = r'[\+\-\*\,;\$><!:\.\|&\^=\(\)\[\]\{\}]+'

In [6]:
decimal_literal = r'\d+i?'           
octal_literal = r'0[1-7]*'
hex_literal = r'0[xX][a-fA-F]+'
floating_literal = r''' \d+\.\d*(?:[eE][+-]\d+)?i?     
                | \d+[eE][+-]\d+i?               
                | \.\d+(?:[eE][+-]\d+)?i?        
                '''
string_literal = r'''\"\s*.*\n?\"'''

In [ ]:
tokenizer.tokenize("")

In [7]:
comments = r'//.*'

In [8]:
patterns =  keywords + '|' + identifier + '|' + operators + '|' \
        + decimal_literal + '|' + octal_literal + '|' + hex_literal + '|' \
        + floating_literal + '|' + string_literal + '|' + comments

In [9]:
tokenizer = RegexpTokenizer(patterns)

In [165]:
annotations = []
for code in codes['Code'][:100]:
    dic = {'text' : code, 'tokens' : [], 'anns' : []}
    code_lines = list(filter(None, code.split('\n')))
    for line in code_lines:
#         print(line)
        tokens = tokenizer.tokenize(line)
#         print(tokens)
        dic['tokens'] += (tokens)
        ann = []
        for t in tokens:
            if re.match(comments, t):
                ann.append('comment')
                t = t.replace('//', '')
                # annotate comments as natural language
                ann += nltk.word_tokenize(t)
            elif re.match(keywords, t):
                ann.append('keyword')
            elif re.match(identifier, t):
                ann.append('identifier')
            elif re.match(operators, t):
                ann.append('operator')
            elif re.match(decimal_literal, t):
                ann.append('decimal_literal')
            elif re.match(octal_literal, t):
                ann.append('octal_literal')
            elif re.match(hex_literal, t):
                ann.append('hex_literal')
            elif re.match(string_literal, t):
                ann.append('string_literal')
            elif re.match(floating_literal, t):
                ann.append('floating_literal')
            else:
                ann.append('undefined')
        dic['anns'] += (ann)
    annotations.append(dic.values())
results = pd.DataFrame(annotations, columns = ['text', 'tokens', 'annotations'])
#         print(ann)
#         input("================")

In [166]:
results

,text,tokens,annotations
0,"package main\n\nimport ""rand""\n\nfunc Routine1...","[package, main, import, ""rand"", func, Routine1...","[keyword, identifier, keyword, string_literal,..."
1,"package main\n\nfunc parseFile(name string, fi...","[package, main, func, parseFile, (, name, stri...","[keyword, identifier, keyword, identifier, ope..."
2,"package hello\n\nimport (\n ""fmt""\n ""net...","[package, hello, import, (, ""fmt"", ""net/http"",...","[keyword, identifier, keyword, operator, strin..."
3,// Copyright 2012 Stefan Nilsson\n//\n// Licen...,"[// Copyright 2012 Stefan Nilsson, //, // Lice...","[comment, Copyright, 2012, Stefan, Nilsson, co..."
4,// Handle the reads\n var tbuf [81920]byte\n\...,"[// Handle the reads, var, tbuf, [, 81920, ], ...","[comment, Handle, the, reads, keyword, identif..."
5,"package main\n\nimport (\n ""fmt""\n ""math...","[package, main, import, (, ""fmt"", ""math/rand"",...","[keyword, identifier, keyword, operator, strin..."
6,"package main\n\nimport (\n ""fmt""\n ""math...","[package, main, import, (, ""fmt"", ""math/rand"",...","[keyword, identifier, keyword, operator, strin..."
7,"package main\n\nimport (\n ""fmt""\n ""math...","[package, main, import, (, ""fmt"", ""math/rand"",...","[keyword, identifier, keyword, operator, strin..."
8,"package main\n\nimport (\n ""fmt""\n ""gith...","[package, main, import, (, ""fmt"", ""github.com/...","[keyword, identifier, keyword, operator, strin..."
9,func Loop(program gl.Uint) {\n start_time :...,"[func, Loop, (, program, gl, ., Uint, ), {, st...","[keyword, identifier, operator, identifier, id..."


In [162]:
results['post_id'] = codes['PostId']

44

In [164]:
results.to_csv('../processed_data/ann_100_results.csv')

In [171]:
for index, result in results.iterrows():
    

0
